<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import-Latest-COVID-19-model-from-EMMAA" data-toc-modified-id="Import-Latest-COVID-19-model-from-EMMAA-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Import Latest COVID-19 model from EMMAA</a></span></li><li><span><a href="#Find-all-INDRA-statements-associated-with-a-publication" data-toc-modified-id="Find-all-INDRA-statements-associated-with-a-publication-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Find all INDRA statements associated with a publication</a></span></li><li><span><a href="#Get-statements-by-subject-and-object" data-toc-modified-id="Get-statements-by-subject-and-object-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Get statements by subject and object</a></span></li><li><span><a href="#Print-bibliography" data-toc-modified-id="Print-bibliography-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Print bibliography</a></span></li></ul></li><li><span><a href="#Get-causal-claims-from-Logette-et-al-2021" data-toc-modified-id="Get-causal-claims-from-Logette-et-al-2021-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get causal claims from Logette et al 2021</a></span></li><li><span><a href="#Get-bibliography-of-Logette-et-al-2021" data-toc-modified-id="Get-bibliography-of-Logette-et-al-2021-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get bibliography of Logette et al 2021</a></span></li><li><span><a href="#Get-subset-of-EMMAA-graph-corresponding-to-bibliography" data-toc-modified-id="Get-subset-of-EMMAA-graph-corresponding-to-bibliography-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get subset of EMMAA graph corresponding to bibliography</a></span></li><li><span><a href="#Visualize-EMMAA-subgraph-of-bibliography" data-toc-modified-id="Visualize-EMMAA-subgraph-of-bibliography-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Visualize EMMAA subgraph of bibliography</a></span></li><li><span><a href="#Summarize-EMMAA-graph-of-bibliography" data-toc-modified-id="Summarize-EMMAA-graph-of-bibliography-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Summarize EMMAA graph of bibliography</a></span></li><li><span><a href="#Causal-BEL-statements-from-each-publication-in-bibliography" data-toc-modified-id="Causal-BEL-statements-from-each-publication-in-bibliography-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Causal BEL statements from each publication in bibliography</a></span></li></ul></div>

ERROR: unknown command "indra" - maybe you meant "index"


In [22]:
from collections import Counter
import os
try:
    import pandas as pd, pybel, indra, requests, bibtexparser
except ImportError:
    !pip  install indra pandas pybel requests bibtexparser
    import pandas as pd, pybel, indra, requests, bibtexparser
import pybel.struct

from pybel.io.jupyter import to_jupyter
import pathlib
from IPython.display import HTML, Markdown, Latex
from pybel.struct.filters import build_pmid_inclusion_filter
from pybel.struct.mutation.induction import get_subgraph_by_pubmed
from indra.assemblers.english.assembler import EnglishAssembler
from indra.assemblers.pybel.assembler import PybelAssembler

from IPython.display import HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

In [3]:
pybel.version.get_version()

'0.15.4'

## Import Latest COVID-19 model from EMMAA

Use the pickled covid graph if it exists,
or download latest indra statements from EMMAA

In [19]:
if os.path.exists('covid19.pickle'):
    emmaa_covid_graph = pybel.from_pickle('covid19.pickle')
else:
    emmaa_covid_graph = pybel.from_emmaa('covid19')
    pybel.to_pickle(emmaa_covid_graph, 'covid19.pickle')


In [20]:
emmaa_covid_graph

Name,covid19
Version,d2c7969d-c77c-44f5-9340-dc975df7eeb7
Authors,INDRA
Nodes,100809
Namespaces,11
Edges,825512
Annotations,9
Citations,115782
Authors,0
Components,750
Warnings,0


## Find all INDRA statements associated with a publication

In [5]:
def get_statements_by_pubmed( pubmed, emmaa_covid_graph=emmaa_covid_graph ):
    statements = []
    for u, v, data in emmaa_covid_graph.edges(data=True):
        if  ('citation' in data) and (
             data['citation']['namespace'] == 'pubmed') and (
             data['citation']['identifier'] ==  pubmed):
                #print(emmaa_covid_graph.edge_to_bel(u, v, data))
                if 'evidence' in data:
                    #print(data['citation']['namespace'], data['citation']['identifier'])
                    #print(data['evidence'])
                    statements.append([u,v,data])
    return statements
        

## Get statements by subject and object

In [6]:
def get_statements_by_subject_and_object( subject, obj, emmaa_covid_graph):
    statements = []
    for u, v, data in emmaa_covid_graph.edges(subject, data=True):
        if (v in obj) or (u in obj):
            if 'evidence' in data:
                statements.append([u,v,data])
    return statements


## Print bibliography

In [7]:
import bibtexparser

def print_bib( bibfile, graph=emmaa_covid_graph, pmid_graph = {}, pmid_indra = {}, english_statements = {} ):
    pmids = get_pmids_of_bibtex(bibfile)
    
    return print_pmids( pmids, graph, pmid_graph, pmid_indra, english_statements )

def get_pmids_of_bibtex(bibfile):
    with open(bibfile) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    pmids = dict([(entry['pmid'], entry['title'])
                    for entry in bib_database.entries
                     if 'pmid' in entry])
    return pmids


def print_pmids( pmids, graph, pmid_graph = {}, pmid_indra = {}, english_statements = {}):    
    
    for pmid in pmids:
        display(Markdown(f'## [{pmids[pmid]}](https://pubmed.ncbi.nlm.nih.gov/{pmid})'))
        if pmid not in pmid_graph:
            pmid_graph[pmid] = get_subgraph_by_pubmed(graph, pmid)
        if pmid not in pmid_indra:
            pmid_indra[pmid] = pybel.to_indra_statements(pmid_graph[pmid])
        if pmid not in english_statements:
            pmid_english = EnglishAssembler(pmid_indra[pmid])
            english_model = pmid_english.make_model()
            english_statements[pmid] = english_model.split('.')
        for i in range(len(pmid_indra[pmid])):
            display(Markdown( f'### {english_statements[pmid][i]}\n'))
            for evidence in pmid_indra[pmid][i].evidence:
                print(evidence.annotations['bel'])
                display(Markdown(f"{evidence.text}"))
    return pmid_graph, pmid_indra, english_statements

def convert_to_bel(claims):
    statements_local = []
    indra_claims_local =  []
    for claim in claims:
        if len(claim['statements_local']) >0:
            pba = PybelAssembler(claim['statements_local'])
            claim['bel_local'] = pba.make_model()
            statements_local += claim['statements_local']
            indra_claims_local.append(claim)
    pba = PybelAssembler(statements_local)
    belgraph_local = pba.make_model()
    return claims, belgraph_local

# Get causal claims from Logette et al 2021

In [8]:
from indra.sources.reach.api import process_pmc
from indra.sources import reach

if os.path.exists('logette.pickle'):
    logette_bel = pybel.from_pickle('logette.pickle')
else:
    rp  = process_pmc('PMC8356061')
    logette_bel = PybelAssembler(rp.statements).make_model()
    pybel.to_pickle(logette_bel, 'logette.pickle')


In [9]:


to_jupyter(logette_bel)

<IPython.core.display.Javascript object>

In [10]:
logette_bel

Name,indra
Version,9453ecff-ad33-4d6f-bf56-8aad3a17fa95
Nodes,40
Namespaces,8
Edges,48
Annotations,6
Citations,1
Authors,0
Components,4
Warnings,0
Network Density,0.0308


# Get bibliography of Logette et al 2021

In [11]:
pmids = get_pmids_of_bibtex('../data/blood-glucose.bib')
len(pmids)

358

# Get subset of EMMAA graph corresponding to bibliography

In [12]:
glucose_covid = get_subgraph_by_pubmed(emmaa_covid_graph, pmids.keys())

# Visualize EMMAA subgraph of bibliography

In [13]:
to_jupyter(glucose_covid)

<IPython.core.display.Javascript object>

# Summarize EMMAA graph of bibliography

In [14]:
glucose_covid

Name,
Version,
Nodes,569
Namespaces,10
Edges,844
Annotations,8
Citations,75
Authors,0
Components,63
Warnings,0
Network Density,0.00261


# Causal BEL statements from each publication in bibliography

In [17]:
import pickle
pmid_graph, pmid_indra, english_statements = {}, {}, {}
if os.path.exists('../data/pmid_graph.pickle'):
    with open('../data/pmid_graph.pickle', 'rb') as p1:
        pmid_graph = pickle.load(p1)
if os.path.exists('../data/pmid_indra.pickle'):
    with open('../data/pmid_indra.pickle', 'rb') as p2:
        pmid_indra = pickle.load(p2)
if os.path.exists('../engish_statements.pickle'):    
    with open('../data/english_statements.pickle', 'rb') as p3:
        english_statements = pickle.load( p3)


In [18]:
pmid_graph, pmid_indra,english_statements = print_bib('../data/logette_references.bib',
                                                      emmaa_covid_graph,
                                                      pmid_graph, 
                                                      pmid_indra,
                                                      english_statements)

with open('../data/pmid_graph.pickle', 'wb') as p1:
    pickle.dump(pmid_graph, p1)
with open('../data/pmid_indra.pickle', 'wb') as p2:
    pickle.dump(pmid_indra, p2)
with open('../data/english_statements.pickle', 'wb') as p3:
    pickle.dump(english_statements, p3)

## [Type 2 alveolar cells are stem cells in adult lung.](https://pubmed.ncbi.nlm.nih.gov/23921127)

## [The intersection between aging and cardiovascular disease.](https://pubmed.ncbi.nlm.nih.gov/22499900)

## [Gammadelta T cells are reduced and rendered unresponsive by hyperglycemia and chronic {TNFalpha} in mouse models of obesity and metabolic disease.](https://pubmed.ncbi.nlm.nih.gov/20625397)

## [Metabolic effects of influenza virus infection in cultured animal cells: Intra- and extracellular metabolite profiling.](https://pubmed.ncbi.nlm.nih.gov/20465796)

## [Angiotensin {II}-induced production of mitochondrial reactive oxygen species: potential mechanisms and relevance for cardiovascular disease.](https://pubmed.ncbi.nlm.nih.gov/22443458)

...

bp(MESH:D000078329 ! Workforce) decreases bp(MESH:D007362 ! "Intensive Care Units")


The pandemic of COVID-19 has caused significant disruption in the intensive care unit ( ICU ) due to inadequate staffing , protective equipment , ventilators , and bed capacity .

## [Lung surfactant protein D ({SP}-D) response and regulation during acute and chronic lung injury.](https://pubmed.ncbi.nlm.nih.gov/23435873)

## [Impaired fasting glucose, blood pressure and cardiovascular disease mortality.](https://pubmed.ncbi.nlm.nih.gov/12364347)

## [Age as independent determinant of glucose tolerance.](https://pubmed.ncbi.nlm.nih.gov/2015973)